In [55]:
import pandas as pd
import os
import torch
from torch.utils.data import Dataset
from skimage import io

In [56]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader 
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [57]:
class image_dataset(Dataset):
  def __init__(self,csv_file,root_dir,transform = None):
    self.annotations = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self,index):
    y_label = int(self.annotations.iloc[index,1])
    img_path = os.path.join(self.root_dir+"/{}".format(y_label),self.annotations.iloc[index,0])
    image = io.imread(img_path)
    y_label = torch.tensor(y_label)
    if self.transform:
      image = self.transform(image)
      p = transforms.Compose([transforms.Scale((256,256))])
      image = p(image)
      
    return (image,y_label)

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install --upgrade kaggle


In [ ]:
# Commented out IPython magic to ensure Python compatibility.
def setup(api):
  import shutil
  ss = api.split()
  folder = ss[-1]
  os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle/"+folder
  print(folder)
  %cd /content/gdrive/My Drive/Kaggle/
  !mkdir $folder
  shutil.copy2("kaggle.json","./"+folder+"/kaggle.json")
  %cd $folder
  !kaggle competitions download -c $folder

setup("kaggle competitions download -c human-age-recognition")
!pwd

In [ ]:
!unzip human-age-recognition.zip

In [58]:
df = image_dataset("train.csv",root_dir="./train/train",transform=transforms.ToTensor())

In [59]:
train_set,test_set = torch.utils.data.random_split(dataset=df,lengths=[10000,df.__len__()-10000])

In [60]:
train_loader = DataLoader(dataset=train_set,batch_size=64,shuffle=True)
test_loader = DataLoader(dataset = test_set,batch_size = 64,shuffle = True)

In [61]:
class CNN(nn.Module):
  def __init__(self,in_channels = 1,num_classes = 10):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(in_channels= in_channels,out_channels = 8,kernel_size =(3,3),stride = (1,1),padding = (1,1))
    self.pool1 = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
    self.conv2 = nn.Conv2d(in_channels= 8,out_channels = 16,kernel_size =(3,3),stride = (1,1),padding = (1,1))
    self.pool2 = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
    self.fc1 = nn.Linear(16*64*64,num_classes)

  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = self.pool1(x)
    x = F.relu(self.conv2(x))
    x = self.pool2(x)
    x = x.reshape(x.shape[0],-1)
    x = self.fc1(x)
    return x

In [62]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

in_channels = 3
num_classes = 7
learning_rate = 0.001
batch_size = 64
num_epochs = 4

In [63]:
model = CNN(in_channels,num_classes).to(device = device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

In [ ]:
for epoch in range(num_epochs):
  for batch_idx,(data,targets) in enumerate(train_loader):
    #get data to cuda if possible 
    data = data.cuda()
    targets = targets.cuda()

    scores = model(data)
    loss = criterion(scores,targets)

    #backward
    optimizer.zero_grad()
    loss.backward()


    #gradient_descent or adam-step
    optimizer.step()

# Check the accuracy for the training step
def check_accuracy(loader,model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x = x.cuda()
      y = y.cuda()

      scores = model(x)
      _,predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)
    print(f' Got {num_correct}/{num_samples} with accuracy ={float(num_correct)/float(num_samples)*100:.2f} ')
    model.train()

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)